### Hypothetical Document Embeddings (HyDE)
At a high level, HyDE is an embedding technique that takes queries, generates a hypothetical answer, and then embeds that generated document and uses that as the final example.

In order to use HyDE, we therefore need to provide a base embedding model, as well as an LLMChain that can be used to generate those documents. By default, the HyDE class comes with some default prompts to use (see the paper for more details on them), but we can also create our own.

In [2]:
from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os 


load_dotenv()
os.getenv("GOOGLE_API_KEY") 
my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [3]:
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

## Call Embedding Model
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# LLM
llm = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [5]:
# Load with `web_search` prompt
embeddings = HypotheticalDocumentEmbedder.from_llm(llm, embedding, "web_search")

In [6]:
# Now we can use it as any embedding class!
result = embeddings.embed_query("Where is the Taj Mahal?")

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

with open("Data/state_of_the_union.txt", encoding = "utf-8") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [12]:
docsearch = Chroma.from_texts(texts, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [13]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


##### RAG Example

In [14]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

## Call Embedding Model
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")


vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embedding)

retriever = vectorstore.as_retriever()

In [17]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM
llm = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'## Task Decomposition for Large Language Model Agents\n\nTask decomposition is a crucial aspect of enabling Large Language Models (LLMs) to effectively act as agents in complex environments. It involves breaking down a high-level, often ambiguous task into a sequence of smaller, more manageable subtasks that the LLM can execute. This process is essential for several reasons:\n\n**1. Bridging the Gap between Language and Action:** LLMs excel at understanding and generating natural language, but they lack the ability to directly interact with the real world. Task decomposition allows us to translate complex, human-understandable instructions into a series of actions that the LLM can perform through APIs or other interfaces.\n\n**2. Handling Complexity and Uncertainty:** Real-world tasks are often multifaceted and involve uncertainty. Decomposing tasks into smaller steps allows the LLM to handle complexity by focusing on one subtask at a time. This also enables the agent to adapt to unfo

In [18]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via 

In [20]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

res = final_rag_chain.invoke({"context":retireved_docs,"question":question})

In [21]:
print(res)

Task decomposition for LLM agents is the process of breaking down large, complex tasks into smaller, more manageable subgoals. This allows the agent to handle complex tasks more efficiently. 

The document highlights three methods for task decomposition:

1. **LLM with simple prompting:**  The LLM is prompted to list steps or subgoals for a given task.
2. **Task-specific instructions:**  The agent is given instructions tailored to the specific task, such as "Write a story outline" for writing a novel.
3. **Human inputs:**  Humans can provide the initial task decomposition, guiding the agent's planning process.

The document also mentions two specific techniques for task decomposition:

* **Chain of Thought (CoT):**  The LLM is instructed to "think step by step" to break down complex tasks into smaller steps.
* **Tree of Thoughts (ToT):**  This extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of potential solutions. 

